In [1]:
from composipy import OrthotropicMaterial, LaminateProperty
from composipy import PlateStructure
import matplotlib



In [2]:
#Ply Material Properties (MPa and mm)
E1 = 60800
E2 = 58250
v12 = 0.07
G12 = 4550
t = 0.21

# Plate Parameters (mm)
a = 360.
b = 360.
m = 10 # discretization in x direction
n = 10 # discretization in y direction


# Stacking sequence
stacking = [45,-45,0,90,45,-45,0,90,45,-45]
stacking += stacking[::-1] # symmetry

In [3]:
# Ply material and laminate property objects

ply_1 = OrthotropicMaterial(E1, E2, v12, G12, t)
l1 = LaminateProperty(stacking, ply_1)



In [4]:
l1

composipy.LaminateProperty
stacking = [45, -45, 0, 90, 45, -45, 0, 90, 45, -45, -45, 45, 90, 0, -45, 45, 90, 0, -45, 45]

In [10]:
import composipy
import composipy.optimize

res = composipy.optimize.maximize_buckling_load(a,b,t,E1,E2,v12,G12,Nxx=1)

In [11]:
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 41.7764326077158
       x: [ 5.000e-05 -1.000e+00]
     nit: 2
     jac: [-1.473e+00  1.264e+01]
    nfev: 6
    njev: 2

In [15]:
res.x

array([ 5.e-05, -1.e+00])

In [2]:
import itertools
import numpy as np
import math

def calculate_cost(path, angle_matrix):
    cost = 0
    for i in range(len(path) - 1):
        cost += math.cos(angle_matrix[path[i]][path[i+1]])
    cost += math.cos(angle_matrix[path[-1]][path[0]])  # To complete the tour
    return cost

def brute_force_tsp(angle_matrix):
    num_cities = len(angle_matrix)
    all_permutations = itertools.permutations(range(num_cities))
    
    best_path = None
    best_cost = float('inf')
    
    for path in all_permutations:
        current_cost = calculate_cost(path, angle_matrix)
        if current_cost < best_cost:
            best_cost = current_cost
            best_path = path
    
    return best_path, best_cost

# Example usage:
np.random.seed(42)  # Pour la reproductibilité

angle_matrix = np.random.rand(10, 10) * 90  # Example matrix with random angles between 0 and pi/2
angle_matrix = (angle_matrix + angle_matrix.T)/2

best_path, best_cost = brute_force_tsp(angle_matrix)
print("Best path:", best_path)
print("Best cost:", best_cost)


Best path: (0, 8, 1, 9, 6, 5, 7, 3, 4, 2)
Best cost: -6.48303190678916


In [16]:
import numpy as np
import random
import math

def calculate_cost(path, angle_matrix):
    cost = 0
    for i in range(len(path) - 1):
        cost += math.cos(angle_matrix[path[i]][path[i+1]])
    cost += math.cos(angle_matrix[path[-1]][path[0]])  # To complete the tour
    return cost

def simulated_annealing(angle_matrix, initial_temp, cooling_rate, stopping_temp):
    num_cities = len(angle_matrix)
    
    # Generate an initial solution
    current_solution = list(range(num_cities))
    random.shuffle(current_solution)
    current_cost = calculate_cost(current_solution, angle_matrix)
    
    best_solution = current_solution.copy()
    best_cost = current_cost
    
    temperature = initial_temp
    
    while temperature > stopping_temp:
        # Create a new solution by swapping two cities
        new_solution = current_solution.copy()
        i, j = random.sample(range(num_cities), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        new_cost = calculate_cost(new_solution, angle_matrix)
        
        # Determine if we should accept the new solution
        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost
            
            # Update the best solution found so far
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        
        # Cool down the temperature
        temperature *= cooling_rate
    
    return best_solution, best_cost

initial_temp = 1000
cooling_rate = 0.99
stopping_temp = 0.00001

best_path, best_cost = simulated_annealing(angle_matrix, initial_temp, cooling_rate, stopping_temp)
print("Best path:", best_path)
print("Best cost:", best_cost)


Best path: [0, 8, 1, 9, 6, 5, 7, 3, 4, 2]
Best cost: -6.48303190678916


In [36]:
import numpy as np
import math
import random

class AntColony:
    def __init__(self, angle_matrix, num_ants, num_iterations, decay, alpha=1, beta=1):
        self.angle_matrix = angle_matrix
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.num_cities = len(angle_matrix)
        self.pheromone = np.ones((self.num_cities, self.num_cities)) / self.num_cities

    def run(self):
        best_path = None
        best_cost = float('inf')
        
        for i in range(self.num_iterations):
            all_paths = self.generate_all_paths()
            self.spread_pheromone(all_paths, self.num_ants)
            best_path, best_cost = self.get_best_path(all_paths, best_path, best_cost)
            print(f'------ Iteration {i} ------')
            print('current best_path :', best_path, 'current best_cost :',best_cost)
            self.pheromone * self.decay
            
        return best_path, best_cost

    def generate_all_paths(self):
        all_paths = []
        for i in range(self.num_ants):
            path = self.generate_path(0)
            all_paths.append((path, self.calculate_cost(path)))
        return all_paths

    def generate_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        
        for _ in range(self.num_cities - 1):
            move = self.pick_move(self.pheromone[prev], self.angle_matrix[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
            
        path.append((prev, start))
        return path

    def pick_move(self, pheromone, angles, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        
        row = pheromone ** self.alpha * ((1.0 / (angles + 1e-10)) ** self.beta)
        norm_row = row / row.sum()
        move = np_choice(range(self.num_cities), 1, p=norm_row)[0]
        return move

    def calculate_cost(self, path):
        total_cost = 0
        for move in path:
            total_cost += math.cos(self.angle_matrix[move[0]][move[1]])
        return total_cost

    def spread_pheromone(self, all_paths, num_ants):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, cost in sorted_paths[:num_ants]:
            for move in path:
                self.pheromone[move] += 1.0 / self.angle_matrix[move[0]][move[1]]

    def get_best_path(self, all_paths, best_path, best_cost):
        for path, cost in all_paths:
            if cost < best_cost:
                best_path = path
                best_cost = cost
        return best_path, best_cost

def np_choice(a, size, p):
    return np.random.choice(a, size=size, p=p)

# Test with a small example matrix
#angle_matrix = np.array([
#    [0, 1, 2, 3],
#    [1, 0, 1, 2],
#    [2, 1, 0, 1],
#    [3, 2, 1, 0]
#])

num_ants = 100
num_iterations = 1000
decay = 0.995
alpha = 0.7
beta = 0.001

ant_colony = AntColony(angle_matrix, num_ants, num_iterations, decay, alpha, beta)
best_path_aco, best_cost_aco = ant_colony.run()

print("Ant Colony Optimization - Best path:", best_path_aco, "Best cost:", best_cost_aco)


------ Iteration 0 ------
current best_path : [(0, 4), (4, 2), (2, 6), (6, 5), (5, 7), (7, 1), (1, 8), (8, 9), (9, 3), (3, 0)] current best_cost : -3.9733174762993335
------ Iteration 1 ------
current best_path : [(0, 4), (4, 2), (2, 6), (6, 5), (5, 7), (7, 1), (1, 8), (8, 9), (9, 3), (3, 0)] current best_cost : -3.9733174762993335
------ Iteration 2 ------
current best_path : [(0, 7), (7, 4), (4, 2), (2, 6), (6, 5), (5, 8), (8, 9), (9, 1), (1, 3), (3, 0)] current best_cost : -4.360444734146218
------ Iteration 3 ------
current best_path : [(0, 7), (7, 4), (4, 2), (2, 6), (6, 5), (5, 8), (8, 9), (9, 1), (1, 3), (3, 0)] current best_cost : -4.360444734146218
------ Iteration 4 ------
current best_path : [(0, 7), (7, 4), (4, 2), (2, 6), (6, 5), (5, 8), (8, 9), (9, 1), (1, 3), (3, 0)] current best_cost : -4.360444734146218
------ Iteration 5 ------
current best_path : [(0, 7), (7, 4), (4, 2), (2, 6), (6, 5), (5, 8), (8, 9), (9, 1), (1, 3), (3, 0)] current best_cost : -4.360444734146218
--

In [1]:
import subprocess



def launch_calcul():
    l1 = subprocess.check_output('wsl -d smeca /opt/sm/bin/as_run /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/export')
    return(l1)

def write_in_aster(chemin):
    filename = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/RunCase_11_Stage_1.comm'
    with open(filename, 'r') as file:
            lines = file.readlines()

    to_bewritten = f'''_F(EPAIS=5.0, 
                                   MATER={liste_mat[chemin[0]]}, 
                                   ORIENTATION={0}),'''
    for i in range(int(len(chemin))-1):
        to_bewritten += f'''\n                                _F(EPAIS=5.0, 
                                   MATER={liste_mat[chemin[i]]}, 
                                   ORIENTATION={antisymmetric_matrix[chemin[i]][chemin[i+1]]}),'''

    for k, line in enumerate(lines):
        if 'clt140 = ' in line:
            index = k
            break
    lines[index] = f'''clt140 = DEFI_COMPOSITE(identifier='5:1', 
                        COUCHE=({to_bewritten}))\n'''

    with open(filename, 'w') as file:
        file.writelines(lines)
    
    with open(filename, 'w') as file:
        file.writelines(lines)
    for i, line in enumerate(lines):
            if 'N = ' in line:
                lines[i] = f'N = {len(chemin)}\n'
                break

    with open(filename, 'w') as file:
        file.writelines(lines)

In [2]:
def remove_section():
        file_path = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/RunCase_11_Stage_1.comm'
        with open(file_path, 'r') as file:
            lines = file.readlines()

        start_keyword = 'clt140 = '
        end_keyword = 'fieldmat = '
        new_lines = []
        inside_section = False

        for line in lines:
            if start_keyword in line:
                new_lines.append(line)
                inside_section = True
            elif end_keyword in line and inside_section:
                inside_section = False
                new_lines.append(line)
            elif not inside_section:
                new_lines.append(line)

        with open(file_path, 'w') as file:
            file.writelines(new_lines)

In [3]:
def read_defo():
        file_path = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/defo'
        defo_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "EPXX" in line:
                    start_reading = True

                if start_reading:
                    if '=====>' in line:
                        break
                    try:
                        defo = line.split("|")
                        if len(defo) <= 5:
                            break
                        else:
                            defo = float(defo[4])
                            defo_values.append(defo)
                    except ValueError:
                        continue
        return defo_values

def read_cons():
        file_path = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/cons'
        defo_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "SIXX" in line:
                    start_reading = True

                if start_reading:
                    if '=====>' in line:
                        break
                    try:
                        defo = line.split("|")
                        if len(defo) <= 5:
                            break
                        else:
                            defo = float(defo[4])
                            defo_values.append(defo)
                    except ValueError:
                        continue
        return defo_values

In [4]:
liste_mat = ['bois1', 'bois2', 'bois1', 'bois2']
import numpy as np

# Créer une matrice 5x5 avec des éléments aléatoires entre -90 et 90
matrix = np.random.randint(-90, 91, size=(4, 4))

# Rendre la matrice antisymétrique en soustrayant sa transpose et en divisant par 2
antisymmetric_matrix = (matrix - matrix.T) / 2

# Mettre les éléments diagonaux à zéro
np.fill_diagonal(antisymmetric_matrix, 0)

In [7]:
import itertools
import numpy as np
import math

def calculate_cost(path):
    remove_section()
    write_in_aster(path)
    
    launch_calcul()
    defo = max(np.abs(read_defo()))
    cons = max(np.abs(read_cons()))
    return(cons/defo)

def brute_force_tsp(angle_matrix):
    num_cities = len(angle_matrix)
    all_permutations = itertools.permutations(range(num_cities))
    
    best_path = None
    best_cost = float('inf')
    
    for path in all_permutations:
        current_cost = calculate_cost(path)
        if current_cost < best_cost:
            best_cost = current_cost
            best_path = path
        print('chemin testé :' , path, 'cout actuel :' , current_cost)
    return best_path, best_cost


best_path, best_cost = brute_force_tsp(antisymmetric_matrix)
print("Best path:", best_path)
print("Best cost:", best_cost)

chemin testé : (0, 1, 2, 3) cout actuel : 5069.585870455439
chemin testé : (0, 1, 3, 2) cout actuel : 7704.746535874708
chemin testé : (0, 2, 1, 3) cout actuel : 8769.392627620235
chemin testé : (0, 2, 3, 1) cout actuel : 9478.29209672251
chemin testé : (0, 3, 1, 2) cout actuel : 8595.303140746737
chemin testé : (0, 3, 2, 1) cout actuel : 10229.778885842972
chemin testé : (1, 0, 2, 3) cout actuel : 7308.010642715174
chemin testé : (1, 0, 3, 2) cout actuel : 5955.980180248793
chemin testé : (1, 2, 0, 3) cout actuel : 6874.9785720238215
chemin testé : (1, 2, 3, 0) cout actuel : 6518.32903088356
chemin testé : (1, 3, 0, 2) cout actuel : 10098.003374489212
chemin testé : (1, 3, 2, 0) cout actuel : 11258.74229815473
chemin testé : (2, 0, 1, 3) cout actuel : 11023.982385411435
chemin testé : (2, 0, 3, 1) cout actuel : 8751.47841401925
chemin testé : (2, 1, 0, 3) cout actuel : 9437.88467136821
chemin testé : (2, 1, 3, 0) cout actuel : 6600.118943525421
chemin testé : (2, 3, 0, 1) cout actuel 

In [18]:
10*9*8*7*6*5*4*3*2*13/60/60

13104.0

In [10]:
import random
def simulated_annealing(angle_matrix, initial_temp, cooling_rate, stopping_temp):
    num_cities = len(angle_matrix)
    
    # Generate an initial solution
    current_solution = list(range(num_cities))
    random.shuffle(current_solution)
    current_cost = calculate_cost(current_solution)
    
    best_solution = current_solution.copy()
    best_cost = current_cost
    
    temperature = initial_temp
    
    while temperature > stopping_temp:
        # Create a new solution by swapping two cities
        new_solution = current_solution.copy()
        i, j = random.sample(range(num_cities), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        new_cost = calculate_cost(new_solution)
        
        # Determine if we should accept the new solution
        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost
            
            # Update the best solution found so far
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        
        # Cool down the temperature
        temperature *= cooling_rate
    
    return best_solution, best_cost

initial_temp = 1000
cooling_rate = 0.99
stopping_temp = 0.00001

best_path, best_cost = simulated_annealing(antisymmetric_matrix, initial_temp, cooling_rate, stopping_temp)
print("Best path:", best_path)
print("Best cost:", best_cost)

KeyboardInterrupt: 

In [19]:
liste_mat = ['bois1', 'bois2', 'bois1', 'bois2','bois1','bois2','bois1','bois2', 'bois1', 'bois2']
import numpy as np

# Créer une matrice 5x5 avec des éléments aléatoires entre -90 et 90
matrix = np.random.randint(-90, 91, size=(10, 10))

# Rendre la matrice antisymétrique en soustrayant sa transpose et en divisant par 2
antisymmetric_matrix = (matrix - matrix.T) / 2

# Mettre les éléments diagonaux à zéro
np.fill_diagonal(antisymmetric_matrix, 0)

In [22]:
import numpy as np
import math
import itertools

# Fonction de calcul du coût (à adapter selon vos besoins)
def calculate_cost(path):
    remove_section()
    write_in_aster(path)
    
    launch_calcul()
    defo = max(np.abs(read_defo()))
    cons = max(np.abs(read_cons()))
    return -cons / defo

# Algorithme des fourmis pour le TSP
def ant_colony_tsp(angle_matrix, num_ants, num_iterations, alpha, beta, evaporation_rate):
    num_cities = len(angle_matrix)
    
    # Initialiser les phéromones
    pheromones = np.ones((num_cities, num_cities))
    best_path = None
    best_cost = float('inf')
    
    for iteration in range(num_iterations):
        all_paths = []
        all_costs = []
        
        for ant in range(num_ants):
            path = [np.random.randint(num_cities)]
            while len(path) < num_cities:
                current_city = path[-1]
                probabilities = []
                for next_city in range(num_cities):
                    if next_city not in path:
                        tau = pheromones[current_city, next_city] ** alpha
                        eta = (1 / angle_matrix[current_city, next_city]) ** beta if angle_matrix[current_city, next_city] != 0 else 1
                        probabilities.append(tau * eta)
                    else:
                        probabilities.append(0)
                
                probabilities = np.array(probabilities)
                probabilities = probabilities / probabilities.sum()
                next_city = np.random.choice(range(num_cities), p=probabilities)
                path.append(next_city)
            
            current_cost = calculate_cost(path)
            all_paths.append(path)
            all_costs.append(current_cost)
            
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = path
            
            print(f'Ant {ant} path: {path}, cost: {current_cost}')
        
        # Mise à jour des phéromones
        for i in range(num_cities):
            for j in range(num_cities):
                pheromones[i, j] *= (1 - evaporation_rate)
                
        for path, cost in zip(all_paths, all_costs):
            for i in range(len(path) - 1):
                pheromones[path[i], path[i + 1]] += 1 / cost
            pheromones[path[-1], path[0]] += 1 / cost  # pour rendre le chemin circulaire
    
    return best_path, best_cost

# Paramètres
num_ants = 10
num_iterations = 10
alpha = 1.0  # Importance de la trace de phéromone
beta = 2.0   # Importance de l'heuristique
evaporation_rate = 0.5

# Création de la matrice angle_matrix
angle_matrix = antisymmetric_matrix

# Appel de l'algorithme des fourmis
best_path, best_cost = ant_colony_tsp(angle_matrix, num_ants, num_iterations, alpha, beta, evaporation_rate)
print("Best path:", best_path)
print("Best cost:", best_cost)

Ant 0 path: [3, 7, 9, 5, 8, 1, 2, 0, 6, 4], cost: -9786.23458845515
Ant 1 path: [2, 8, 5, 9, 7, 1, 6, 3, 4, 0], cost: -10439.4621270467
Ant 2 path: [0, 3, 7, 9, 5, 6, 1, 8, 4, 2], cost: -8413.656946831856
Ant 3 path: [2, 3, 7, 9, 5, 6, 1, 8, 4, 0], cost: -9867.1241709099
Ant 4 path: [7, 3, 4, 9, 5, 8, 1, 6, 0, 2], cost: -10742.754540990689
Ant 5 path: [5, 9, 4, 7, 3, 6, 8, 1, 2, 0], cost: -10271.083026890632
Ant 6 path: [1, 7, 3, 0, 2, 6, 8, 5, 9, 4], cost: -10740.807070191799
Ant 7 path: [7, 3, 6, 5, 9, 4, 8, 1, 2, 0], cost: -10339.675220151365
Ant 8 path: [4, 6, 0, 2, 5, 9, 3, 7, 1, 8], cost: -10828.899004462342
Ant 9 path: [8, 5, 9, 0, 3, 7, 1, 2, 6, 4], cost: -7447.883614998058
Ant 0 path: [3, 7, 4, 9, 5, 6, 8, 0, 2, 1], cost: -8614.89935638716
Ant 1 path: [8, 1, 6, 0, 3, 7, 9, 5, 2, 4], cost: -10559.720712533199
Ant 2 path: [7, 0, 2, 9, 5, 8, 1, 4, 3, 6], cost: -10005.34464220605
Ant 3 path: [9, 5, 8, 6, 7, 3, 4, 0, 2, 1], cost: -9071.028452465605
Ant 4 path: [6, 5, 9, 3, 7, 0, 4,

In [26]:
for i in range (len(best_path)):
    print(liste_mat[best_path[i]])

bois1
bois2
bois2
bois1
bois2
bois2
bois1
bois1
bois2
bois1


In [217]:
def write_in_aster(chemin):
    filename = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/RunCase_11_Stage_1.comm'
    with open(filename, 'r') as file:
            lines = file.readlines()

    to_bewritten = f'''_F(EPAIS=5.0, 
                                   MATER= bois1, 
                                   ORIENTATION={chemin[0]}),'''
    for i in range(int(len(chemin))-1):
        if i%2==0:
            to_bewritten += f'''\n                                _F(EPAIS=5.0, 
                                    MATER=bois1, 
                                    ORIENTATION={chemin[i]}),'''
        else :
            to_bewritten += f'''\n                                _F(EPAIS=5.0, 
                                    MATER=bois2, 
                                    ORIENTATION={chemin[i]}),''' 

    for k, line in enumerate(lines):
        if 'clt140 = ' in line:
            index = k
            break
    lines[index] = f'''clt140 = DEFI_COMPOSITE(identifier='5:1', 
                        COUCHE=({to_bewritten}))\n'''

    with open(filename, 'w') as file:
        file.writelines(lines)
    for i, line in enumerate(lines):
            if 'N = ' in line:
                lines[i] = f'N = {len(chemin)}\n'
                break

    with open(filename, 'w') as file:
        file.writelines(lines)

In [218]:
import random
from deap import base, creator, tools, algorithms

# Définir la fonction de calcul de rigidité
def calculate_rigidity(configuration):
    remove_section()
    write_in_aster(configuration)
    
    launch_calcul()
    defo = max(np.abs(read_defo()))
    cons = max(np.abs(read_cons()))
    res = cons / defo
    print(res)
    return res,

# Configuration GA
N_max = 20  # Nombre maximal de plis
orientations = [0, 45, 90, -45]

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def init_individual():
    n_plies = random.randint(3, N_max)  # S'assurer qu'il y a au moins 1 pli
    return creator.Individual([random.choice(orientations) for _ in range(n_plies)])

toolbox = base.Toolbox()
toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", calculate_rigidity)

def cx_custom(ind1, ind2):
    if len(ind1) > 1 and len(ind2) > 1:
        return tools.cxTwoPoint(ind1, ind2)
    else:
        return ind1, ind2

def mut_individual(individual):
    if len(individual) > 1:
        index = random.randrange(len(individual))
        individual[index] = random.choice(orientations)
    else:
        # Si l'individu n'a qu'un pli, il faut simplement changer son orientation
        individual[0] = random.choice(orientations)
    return individual,

toolbox.register("mate", cx_custom)
toolbox.register("mutate", mut_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

# Paramètres GA
population = toolbox.population(n=10)
ngen = 10
cxpb = 0.5
mutpb = 0.2

# Exécuter l'algorithme génétique
result, log = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=False)

# Meilleure solution trouvée
best_individual = tools.selBest(population, k=1)[0]
print("Meilleure configuration de plis:", best_individual)
print("Rigidité maximale:", calculate_rigidity(best_individual)[0])


3452.49945513882
8672.103824715607
5330.879744655067
4452.29140090506
9058.486938920172
6901.037293583345
9076.6822090376
1625.377190949856
2615.2613073769057
8276.103249742811
8310.389299365224
5261.465229885726
9058.486938920172
9016.465084563406
5440.501603100427
5273.999465416351
9145.348164920953
8735.12975329279
9235.198141190947
8904.772271620628
9145.348164920953
9076.6822090376
8895.914044779784
9784.145468491839
9055.992140853989
9145.348164920953
9145.348164920953
9068.074682501243
9442.870868262866
9145.348164920953
9466.548875929582
9145.348164920953
9442.870868262866
9145.348164920953
9442.870868262866
9374.858380912598
9386.450883425254
9145.348164920953
8580.141291718453
9794.083858501746
9442.870868262866
9112.545749106734
9794.083858501746
9442.870868262866
9442.870868262866
9442.870868262866
9794.083858501746
9442.870868262866
9794.083858501746
9794.083858501746
9794.083858501746
9794.083858501746
9442.870868262866
9442.870868262866
9794.083858501746
9794.08385850174